In [ ]:
import random
from hashlib import sha256


def is_prime(n: int, k: int = 40) -> bool:
    """Miller-Rabin primality test"""
    if n < 2 or n % 2 == 0: return False
    if n == 2 or n == 3: return True
    
    # Phân tích n-1 = 2^r * d
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2
    
    # Vòng lặp kiểm tra (Witness loop)
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, d, n)
        
        if x == 1 or x == n - 1:
            continue
        
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True


def generate_prime(bit_length: int, k: int = 40) -> int:
    """
    Sinh số nguyên tố ngẫu nhiên với độ dài bit chỉ định.
    Sử dụng Miller-Rabin với k rounds để tăng xác suất chính xác.
    """
    while True:
        num = random.getrandbits(bit_length)
        num |= (1 << bit_length - 1) | 1 
        if is_prime(num, k):
            return num


def prime_factors(n: int) -> list:
    """
    Phân tích thừa số nguyên tố của n (tối ưu).
    Chỉ duyệt tới sqrt(n), không duyệt hết đến n.
    """
    factors = []
    d = 2
    
    # Kiểm tra 2
    while n % 2 == 0:
        if not factors or factors[-1] != 2:
            factors.append(2)
        n //= 2
    
    # Kiểm tra các số lẻ từ 3 đến sqrt(n)
    d = 3
    while d * d <= n:
        while n % d == 0:
            if not factors or factors[-1] != d:
                factors.append(d)
            n //= d
        d += 2
    
    # Nếu n > 1, thì n là thừa số nguyên tố
    if n > 1:
        factors.append(n)
    
    return factors


def find_primitive_root(p: int) -> int:
    if p == 2: 
        return 1
    
    # Phân tích thừa số nguyên tố của p-1
    phi = p - 1
    factors = prime_factors(phi)
    
    # Thử các ứng viên g từ 2 trở đi
    # Chỉ cần thử khoảng 100-1000 ứng viên (rất hiếm khi phải thử nhiều)
    for g in range(2, min(p, 10000)):
        is_primitive = True
        
        # Kiểm tra: g^((p-1)/factor) mod p != 1 với mọi thừa số
        for factor in factors:
            if pow(g, phi // factor, p) == 1:
                is_primitive = False
                break
        
        if is_primitive:
            return g
    
    # Fallback: Nếu không tìm được trong 10000 ứng viên, trả về 2 (hiếm xảy ra)
    return 2


def mod_inverse(a: int, m: int) -> int:
    """Tính nghịch đảo modulo a^-1 mod m (Extended Euclidean Algorithm)."""
    def extended_gcd(a, b):
        if a == 0:
            return b, 0, 1
        gcd_val, x1, y1 = extended_gcd(b % a, a)
        x = y1 - (b // a) * x1
        y = x1
        return gcd_val, x, y
        
    _, x, _ = extended_gcd(a % m, m)
    return (x % m + m) % m


def hash_message(message: bytes, q: int) -> int:
    """Băm tin nhắn thành một số nguyên modulo q (SHA-256)."""
    hash_obj = sha256(message)
    hash_bytes = hash_obj.digest()
    hash_int = int.from_bytes(hash_bytes, byteorder='big')
    return hash_int % q


class Schnorr:

    def __init__(self, bit_length: int = 512):
        """
        Khởi tạo Schnorr với sinh khóa.
        LƯU Ý: Dùng 512 bit cho tốc độ test, 1024+ cho bảo mật thực tế.
        """
        self.bit_length = bit_length
        self.p = None      # Large prime (modulus)
        self.q = None      # Subgroup order (prime factor of p-1)
        self.g = None      # Generator of subgroup
        self.x = None      # Private key (secret)
        self.y = None      # Public key (y = g^x mod p)
        
        self._generate_keys()
        
    def _generate_keys(self) -> None:
        """
        Sinh cặp khóa Schnorr (FIX: Sử dụng phương pháp Safe Prime).
        Safe Prime: p = 2q + 1 (q là nguyên tố) => dễ tìm generator
        """

        # PHƯƠNG PHÁP SAFE PRIME (Tối ưu và nhanh hơn)
        # Sinh q, sau đó sinh p = 2q + 1 (p là safe prime)
        
        attempt = 0
        max_attempts = 1000
        
        while attempt < max_attempts:
            # 1. Sinh q (nhỏ hơn, sinh nhanh)
            q_candidate = generate_prime(self.bit_length - 1, k=40)
            
            # 2. Kiểm tra p = 2q + 1 có phải nguyên tố không
            p_candidate = 2 * q_candidate + 1
            
            if is_prime(p_candidate, k=40):
                self.q = q_candidate
                self.p = p_candidate
                print(f"✓ Generated q ({self.bit_length - 1}-bit) and p = 2q + 1 ({self.bit_length}-bit)")
                break
            
            attempt += 1
            if attempt % 10 == 0:
                print(f"   Searching for safe prime pair... (attempt {attempt}/{max_attempts})")
        
        if attempt >= max_attempts:
            raise RuntimeError(f"Lỗi: Không sinh được prime sau {max_attempts} lần thử")
        
        # 3. Tìm Generator g
        # Với safe prime p = 2q + 1, generator là phần tử bậc q
        # g = 2 hoặc g = p - 1 thường là generator (nếu 2^q mod p != 1)
        print(f"   ⏳ Finding generator g...")
        
        # Thử g = 2 trước
        if pow(2, self.q, self.p) != 1:
            self.g = 2
        # Nếu không, thử g = p - 1 (cách khác: -1 mod p)
        elif pow(self.p - 1, self.q, self.p) != 1:
            self.g = self.p - 1
        else:
            # Thử các số ngẫu nhiên
            for _ in range(1000):
                g_candidate = random.randint(2, self.p - 2)
                if pow(g_candidate, self.q, self.p) != 1 and pow(g_candidate, 2, self.p) != 1:
                    self.g = g_candidate
                    break
        
        if self.g is None:
            raise RuntimeError(" Lỗi: Không tìm được generator")
        
        print(f" Found generator g: {self.g}")
        
        # 4. Sinh khóa bí mật x
        self.x = random.randint(1, self.q - 1)
        
        # 5. Tính khóa công khai y = g^x mod p
        self.y = pow(self.g, self.x, self.p)

    def get_public_key(self) -> tuple:
        """Trả về khóa công khai (p, q, g, y)"""
        return (self.p, self.q, self.g, self.y)

    def get_private_key(self) -> int:
        """Trả về khóa bí mật x"""
        return self.x

    def sign(self, message: bytes) -> tuple:
        """
        Tạo chữ ký Schnorr cho tin nhắn M.
        Returns: Chữ ký (h, s)
        """
        # 1. Chọn random nonce k
        k = random.randint(1, self.q - 1)
        
        # 2. Tính t = g^k mod p
        t = pow(self.g, k, self.p)
        
        # 3. Tính Challenge h = H(M || t) mod q
        t_bytes = t.to_bytes((t.bit_length() + 7) // 8, byteorder='big')
        message_with_t = message + t_bytes
        h = hash_message(message_with_t, self.q)
        
        # 4. Tính Response s = (k + h*x) mod q
        s = (k + h * self.x) % self.q
        
        return h, s

    def verify(self, message: bytes, signature: tuple, public_key: tuple = None) -> bool:
        """
        Xác thực chữ ký (h, s).
        """
        if public_key is None: 
            public_key = self.get_public_key()
        
        p, q, g, y = public_key
        h, s = signature

        # 1. Kiểm tra giới hạn
        if not (0 < h < q and 0 < s < q): 
            return False

        # 2. Tính t' = g^s * y^(-h) mod p
        y_inv_h = pow(y, q - h, p) 
        t_prime = (pow(g, s, p) * y_inv_h) % p 

        # 3. Tính h' = H(M || t') mod q
        t_prime_bytes = t_prime.to_bytes((t_prime.bit_length() + 7) // 8, byteorder='big')
        message_with_t = message + t_prime_bytes
        h_prime = hash_message(message_with_t, q)

        # 4. Kiểm tra h' == h
        return h_prime == h


# --- TEST DRIVE ---
if __name__ == '__main__':
    print("=" * 60)
    print("SCHNORR SIGNATURE - TEST DRIVE")
    print("=" * 60 + "\n")
    
    # THIẾT LẬP KHÓA (512 bit cho tốc độ)
    sch_system = Schnorr(bit_length=512)
    
    # 1. SINH KHÓA
    p, q, g, y = sch_system.get_public_key()
    print(f"📋 Public Key Components:")
    print(f"   p (prime): {p}")
    print(f"   q (order): {q}")
    print(f"   g (generator): {g}")
    print(f"   y (public): {y}\n")
    
    # 2. THÔNG ĐIỆP
    message = b"TRANDUCLINH"
    print(f"💬 Message: {message.decode()}\n")
    
    # 3. KÝ
    print(f"🔐 Signing...")
    signature = sch_system.sign(message)
    h, s = signature
    print(f"   Signature (h, s):")
    print(f"   h = {h}")
    print(f"   s = {s}\n")
    
    # 4. XÁC THỰC HỢP LỆ
    print(f"✅ Verifying valid signature...")
    is_valid = sch_system.verify(message, signature)
    print(f"   Result: {is_valid} (Expected: True)\n")
    
    # 5. XÁC THỰC TIN NHẮN GIẢ
    print(f"❌ Verifying FORGED message...")
    hacked_message = b"GIAOTHONGVANTAI!"
    is_invalid = sch_system.verify(hacked_message, signature)
    print(f"   Result: {is_invalid} (Expected: False)\n")
    
    print("=" * 60)
    print("✅ TEST COMPLETED!")
    print("=" * 60)

SCHNORR SIGNATURE - TEST DRIVE

🔑 Starting key generation with 512 bits...
   ⏳ Searching for safe prime pair... (attempt 10/1000)
   ⏳ Searching for safe prime pair... (attempt 20/1000)
   ⏳ Searching for safe prime pair... (attempt 30/1000)
   ⏳ Searching for safe prime pair... (attempt 40/1000)
   ⏳ Searching for safe prime pair... (attempt 50/1000)
   ⏳ Searching for safe prime pair... (attempt 60/1000)
   ⏳ Searching for safe prime pair... (attempt 70/1000)
   ⏳ Searching for safe prime pair... (attempt 80/1000)
   ⏳ Searching for safe prime pair... (attempt 90/1000)
   ⏳ Searching for safe prime pair... (attempt 100/1000)
   ⏳ Searching for safe prime pair... (attempt 110/1000)
   ⏳ Searching for safe prime pair... (attempt 120/1000)
   ⏳ Searching for safe prime pair... (attempt 130/1000)
   ⏳ Searching for safe prime pair... (attempt 140/1000)
   ⏳ Searching for safe prime pair... (attempt 150/1000)
   ⏳ Searching for safe prime pair... (attempt 160/1000)
   ⏳ Searching for saf